In [77]:
from aocd import get_data, submit
import numpy as np
import sys
import re
import math
np.set_printoptions(threshold=sys.maxsize)
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from queue import PriorityQueue
from collections import defaultdict, Counter
from dataclasses import dataclass, field
from enum import Enum
from typing import Dict, List, Tuple, Optional
from functools import reduce, cache
from operator import mul
from bisect import bisect_right

DIRECTIONS_4 = [(-1, 0), (0, 1), (1, 0), (0, -1)]
DIRECTIONS_8 = [(x, y) for x in [1, 0, -1] for y in [1, 0, -1] if not (x ==0 and y == 0)]

def raw_read_input(day, hardcoded_input=None):
    return get_data(day=day, year=2024, block=True) if not hardcoded_input else hardcoded_input

def read_input(day, dtype=None, hardcoded_input=None):
    lines = raw_read_input(day=day, hardcoded_input=hardcoded_input).splitlines()
    if dtype is not None:
        lines = [dtype(x) if x else None for x in lines]
    return lines
    
def read_matrix(day, dtype=np.int32, hardcoded_input=None):
    lines = read_input(day, hardcoded_input=hardcoded_input)
    lines = [[dtype(x) for x in line] for line in lines]
    return np.array(lines, dtype=dtype)

# Day 1

In [2]:
lines = read_input(day=1)
list_a, list_b = zip(*[map(int, pair.split()) for pair in lines])
sorted_a, sorted_b = sorted(list_a), sorted(list_b)
difference = sum(abs(a - b) for a, b in zip(sorted_a, sorted_b))
print('part 1:', difference)

counter_b = Counter(list_b)
similarity = sum(a * counter_b[a] for a in list_a)
print('part 2:', similarity)

part 1: 1666427
part 2: 24316233


# Day 2

In [3]:
reports = read_input(day=2)
def is_safe(levels):
    pairs = list(zip(levels, levels[1:]))
    if not all(1 <= abs(a - b) <= 3 for a, b in pairs):
        return False
    cmp_levels = [a > b for a, b in pairs]
    return all(cmp_levels) or not any(cmp_levels)

def is_safe_with_tolerance(levels):
    return any(is_safe(levels[:index] + levels[index + 1:]) for index in range(len(levels)))
    
levels_report = [list(map(int, report.split())) for report in reports]
print('part 1', sum(is_safe(report) for report in levels_report))
print('part 2', sum(is_safe_with_tolerance(report) for report in levels_report))

part 1 321
part 2 386


# Day 3

In [4]:
full_instructions = raw_read_input(day=3)
all_matches = list(re.findall(r"(don?'?t?\(\))|(mul\(\d{1,3},\d{1,3}\))", full_instructions))

part_1_prod = 0
part_2_prod = 0
active = True
for do_match, mul_match in all_matches:
    if mul_match:
        part_1_prod += eval(mul_match)
        if active:
            part_2_prod += eval(mul_match)
    elif do_match == 'do()':
        active = True
    else:
        active = False
    
print('part 1', part_1_prod)
print('part 2', part_2_prod)

part 1 189527826
part 2 63013756


# Day 4

In [5]:
TARGET = 'XMAS'
matrix = read_matrix(day=4, dtype=str)
padded_matrix = np.pad(matrix, pad_width=((1, 1), (1, 1)))

def dfs(y, x, current_index, visited, directions):
    if TARGET[current_index] != padded_matrix[y, x]:
        return 0
    if current_index == len(TARGET) - 1:
        return 1
    visited = visited | {(y, x)}
    local_sum = 0
    for dy, dx in directions:
        new_y, new_x = y + dy, x + dx
        if (new_y, new_x) in visited:
            continue
        local_sum += dfs(new_y, new_x, current_index + 1, visited, directions)
    return local_sum

found = 0
for i in range(padded_matrix.shape[0]):
    for j in range(padded_matrix.shape[1]):
        for direction in DIRECTIONS_8:
            found += dfs(i, j, 0, set(), [direction])
print('part 1', found)

found = 0
for i in range(padded_matrix.shape[0]):
    for j in range(padded_matrix.shape[1]):
        if (padded_matrix[i, j] == 'A' and 
            {padded_matrix[i - 1, j - 1], padded_matrix[i + 1, j + 1]} == {'M', 'S'} and
            {padded_matrix[i - 1, j + 1], padded_matrix[i + 1, j - 1]} == {'M', 'S'}
        ):
            found += 1
print('part 2', found)

part 1 2718
part 2 2046


# Day 5

In [ ]:
from functools import cmp_to_key
from math import ceil

lines = read_input(day=5)
line_divider_index = lines.index('')
rules, updates = lines[:line_divider_index], lines[line_divider_index + 1:]

less_than = set()
for rule in rules:
    start, end = map(int, rule.split('|'))
    less_than.add((start, end))

def cmp(a, b):
    if (a, b) in less_than:
        return -1
    if (b, a) in less_than:
        return 1
    print(a, b)
    return 0

sum_middle_1 = 0
sum_middle_2 = 0
for update in updates:
    items = list(map(int, update.split(',')))
    sorted_items = sorted(items, key=cmp_to_key(cmp))
    middle = sorted_items[ceil(len(sorted_items) / 2) - 1]
    if items == sorted_items:
        sum_middle_1 += middle
    else:
        sum_middle_2 += middle

print('part 1', sum_middle_1)
print('part 2', sum_middle_2)

# Day 6

In [9]:
from tqdm import tqdm
hardcoded_input = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""

base_matrix = read_matrix(day=6, dtype=str, hardcoded_input=None)
DIRECTIONS = [(-1, 0), (0, 1), (1, 0), (0, -1)]

part_1_answer = None
part_2_answer = 0

for change_y in tqdm(range(base_matrix.shape[0])):
    for change_x in range(base_matrix.shape[1]):
        if base_matrix[change_y, change_x] != '.':
            continue
            
        matrix = base_matrix.copy()
        matrix[change_y, change_x] = '#'

        y, x = [pos[0] for pos in np.where(matrix == '^')]
        direction_index = 0
        visited = set()
        visited_with_direction = set()

        while True:
            visited.add((y, x))
            if (y, x, direction_index) in visited_with_direction:
                part_2_answer += 1
                break
            visited_with_direction.add((y, x, direction_index))
        
            
            dy, dx = DIRECTIONS[direction_index]
            new_y, new_x = dy + y, dx + x
            if not (0 <= new_y < matrix.shape[0] and 0 <= new_x < matrix.shape[1]):
                if part_1_answer is None:
                    part_1_answer = len(visited)
                break
            next_turn_index = (direction_index + 1) % 4
            if matrix[new_y, new_x] == '#':
                direction_index = next_turn_index
                continue
            
            y, x = new_y, new_x
print('part 1', part_1_answer)
print('part 2', part_2_answer)


65%|████████████████████████████████████████████████████████████████████████████████████████████▏                                                | 85/130 [00:34<00:18,  2.44it/s]

KeyboardInterrupt: 

# Day 7

In [21]:
lines = read_input(day=7)
def is_possible(target, current, remaining, with_concat):
    if current > target:
        return False
    if len(remaining) == 0:
        return current == target
    
    return (
            is_possible(target, current * remaining[0], remaining[1:], with_concat) or 
            is_possible(target, current + remaining[0], remaining[1:], with_concat) or
            (with_concat and is_possible(target, int(str(current) + str(remaining[0])), remaining[1:], with_concat))
    )
    
part_1 = 0
part_2 = 0
for equation in lines:
    target_str, values_str = equation.split(': ')
    target, values = int(target_str), list(map(int, values_str.split()))
    if is_possible(target, values[0], values[1:], with_concat=False):
        part_1 += target
    if is_possible(target, values[0], values[1:], with_concat=True):
        part_2 += target
print('part 1', part_1)
print('part 2', part_2)

part 1 4998764814652
part 2 37598910447546


# Day 8

In [2]:
antenna_map = read_matrix(day=8, dtype=str)
antenas = np.where(antenna_map != '.')
antenas = list(zip(antenas[0], antenas[1]))

anti_nodes = set()
anti_nodes_part_1 = set()
for index, (ya, xa) in enumerate(antenas):
    for (yb, xb) in antenas[index + 1:]:
        if antenna_map[ya, xa] != antenna_map[yb, xb]:
            continue
        dy, dx = yb - ya, xb - xa
        
        for distance in range(max(antenna_map.shape)):
            anti_node_1 = ya - dy * distance, xa - dx * distance
            anti_node_2 = yb + dy * distance, xb + dx * distance
            
            for anti_node_pos in [anti_node_1, anti_node_2]:
                if not (0 <= anti_node_pos[0] < antenna_map.shape[0] and 0 <= anti_node_pos[1] < antenna_map.shape[1]):
                    continue
                anti_nodes.add(anti_node_pos)
                if distance == 1:
                    anti_nodes_part_1.add(anti_node_pos)
                    
print('part 1:', len(anti_nodes_part_1))
print('part 2:', len(anti_nodes))

part 1: 291
part 2: 1015


# Day 9

In [3]:
disk_map = list(map(int, raw_read_input(day=9)))
disk = []
for index, batch_size in enumerate(disk_map):
    disk += [index // 2 if index % 2 == 0 else -1] * batch_size
disk = np.array(disk)
empty_count = len(disk[disk == -1]) 
disk[disk == -1] = disk[disk != -1][::-1][:empty_count]
disk = disk[:len(disk) - empty_count]
checksum = sum(index * value for index, value in enumerate(disk))
print('part 1:', checksum)

disk = []
batch_ranges = {}
for index, batch_size in enumerate(disk_map):
    disk += [index // 2 if index % 2 == 0 else -1] * batch_size
    if index % 2 == 0:
        batch_ranges[index // 2] = (len(disk) - batch_size, len(disk) - 1)
disk = np.array(disk)

for index in range(len(disk_map) // 2, 0, -1):
    start, end = batch_ranges[index]
    empties = np.where(disk[:start - (end - start)] == -1)[0]
    for search_index in empties:
        if np.all(disk[search_index: search_index + end - start + 1] == -1):
            disk[search_index: search_index + end - start + 1] = index
            disk[start: end + 1] = -1
            break
disk[disk == -1] = 0
checksum = sum(index * value for index, value in enumerate(disk))
print('part 2:', checksum)

part 1: 6607511583593
part 2: 6636608781232


# Day 10

In [4]:
topological_map = read_matrix(day=10, dtype=int)
starts = np.where(topological_map == 0)
starts = list(zip(starts[0], starts[1]))

def dfs(y, x, current_index, path_so_far, collected_paths, collected_ends):
    path_so_far = path_so_far + [(y, x)]
    if current_index == 9:
        collected_ends.add((y, x))
        collected_paths.add(tuple(path_so_far))
    for dy, dx in DIRECTIONS_4:
        new_y, new_x = y + dy, x + dx
        if not (0 <= new_y < topological_map.shape[0] and 0 <= new_x < topological_map.shape[1]):
            continue
        if topological_map[new_y, new_x] == current_index + 1:
            dfs(new_y, new_x, current_index + 1, path_so_far, collected_paths, collected_ends)

part_1_score = 0
part_2_score = 0
for start_y, start_x in starts:
    collected_ends = set()
    collected_paths = set()
    dfs(start_y, start_x, 0, [], collected_paths, collected_ends)
    part_1_score += len(collected_ends)
    part_2_score += len(collected_paths)
print('part 1:', part_1_score)
print('part 2:', part_2_score)

part 1: 459
part 2: 1034


# Day 11

In [29]:
from functools import cache
from collections import Counter
stones = list(map(int, raw_read_input(day=11).split(' ')))

@cache
def blink_result(stone: int) -> list[int]:
    if stone == 0:
        return [1]
    digits_count = len(str(stone))
    if digits_count % 2 == 0:
        splitter = 10 ** (digits_count // 2)
        return [stone // splitter, stone % splitter]
    return [stone * 2024]

stone_counts = Counter(stones)
for index in range(75):
    new_stones = {stone: blink_result(stone) for stone in stone_counts.keys()}
    new_stone_counts = defaultdict(int)
    for stone, result_stones in new_stones.items():
        for new_stone in result_stones:
            new_stone_counts[new_stone] += stone_counts[stone]
    stone_counts = new_stone_counts
    if index == 24:
        print('part 1:', sum(x for x in stone_counts.values()))
    
print('part 2:', sum(x for x in stone_counts.values()))

part 1: 189547
part 2: 224577979481346


# Day 12

In [81]:
import cv2
plots = read_matrix(day=12, dtype=str, hardcoded_input=None)
plot_names = np.unique(plots)
plots = np.pad(plots, pad_width=((1, 1), (1, 1)), constant_values='.')
all_plots = []
for plot_name in plot_names:
    mask = plots == plot_name
    component_count, labels, _, _ = cv2.connectedComponentsWithStats(mask.astype(np.uint8), connectivity=4)
    for component in range(1, component_count):
        component_coords = np.where(labels == component)
        points = list(zip(component_coords[0], component_coords[1]))
        all_plots.append(points)

def plot_perimeter(points):
    total = 0
    for point in points:
        for dy, dx in DIRECTIONS_4:
            new_y, new_x = point[0] + dy, point[1] + dx
            if plots[new_y, new_x] != plots[point[0], point[1]]:
                total += 1
    return total

def plot_count_sides_of_shape(points):
    total = 0
    for point in points:
        for dir_index, (dy, dx) in enumerate(DIRECTIONS_4):
            new_y, new_x = point[0] + dy, point[1] + dx
            next_dir_index = (dir_index + 1) % 4
            buddy_dy, buddy_dx = DIRECTIONS_4[next_dir_index]
            buddy_y, buddy_x = point[0] + buddy_dy, point[1] + buddy_dx
            buddy_next_y, buddy_next_x = buddy_y + dy, buddy_x + dx
            if plots[new_y, new_x] != plots[point[0], point[1]] and (plots[buddy_y, buddy_x] != plots[point[0], point[1]] or plots[buddy_next_y, buddy_next_x] == plots[point[0], point[1]]):
                total += 1
    return total
    
print('part 1:', sum(len(plot) * plot_perimeter(plot) for plot in all_plots))
print('part 2:', sum(len(plot) * plot_count_sides_of_shape(plot) for plot in all_plots))

part 1: 1359028
part 2: 839780


# Day 13

# Day 14

# Day 15

# Day 16

# Day 17

# Day 18

# Day 19

# Day 20

# Day 21

# Day 22

# Day 23

# Day 24

# Day 25